# Inferencia con LLM (Mistral-3-8B-Instruct)

Este cuaderno realiza inferencia de clasificación binaria (YES/NO) usando un LLM mediante prompting

## Importar librerías

In [1]:
import os
import torch
import pandas as pd
import numpy as np
from dotenv import load_dotenv

from transformers import Mistral3ForConditionalGeneration, FineGrainedFP8Config
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, SystemMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support
)

from pyevall.evaluation import PyEvALLEvaluation
from pyevall.metrics.metricfactory import MetricFactory

# Cargar variables de entorno desde .env
load_dotenv('.env')
# Si no funciona, intentar con ruta relativa al notebook
if not os.getenv("HF_TOKEN"):
    load_dotenv()


In [2]:
# Cargar el token de HuggingFace desde .env
HF_TOKEN = os.getenv("HF_TOKEN", "")
if HF_TOKEN:
    print("✓ HF_TOKEN cargado desde .env")
else:
    print("⚠ HF_TOKEN no encontrado en .env - Continuando sin token")


✓ HF_TOKEN cargado desde .env


## Configuración y parámetros

In [3]:
MODEL_NAME = "mistralai/Ministral-3-8B-Instruct-2512"
MAIN_PATH = ".."
GROUP_ID = "BeingChillingWeWillWin"

TEXT_COLUMN = "tweet"
LABEL_COLUMN = "task1"

DATA_TRAIN_PATH = os.path.join(MAIN_PATH, "preprocessed_data", "train_preprocessed_v2.json")
DATA_VAL_PATH   = os.path.join(MAIN_PATH, "preprocessed_data", "val_preprocessed_v2.json")
DATA_TEST_PATH  = os.path.join(MAIN_PATH, "preprocessed_data", "test_preprocessed_v2.json")

PREDICTIONS_DIR = os.path.join(MAIN_PATH, "results_v2", "Ministral3B", "predictions")
os.makedirs(PREDICTIONS_DIR, exist_ok=True)

## Carga y preprocesamiento de datos

In [4]:
train_df = pd.read_json(DATA_TRAIN_PATH)
val_df   = pd.read_json(DATA_VAL_PATH)
test_df  = pd.read_json(DATA_TEST_PATH)

label_map = {"NO": 0, "YES": 1}
label_map_inverse = {0: "NO", 1: "YES"}

train_df["label"] = train_df[LABEL_COLUMN].map(label_map)
val_df["label"]   = val_df[LABEL_COLUMN].map(label_map)

print(f"Text column used: {TEXT_COLUMN}")
print(f"Train size: {len(train_df)} | Val size: {len(val_df)} | Test size: {len(test_df)}")
print(f"\nDistribución de etiquetas en TRAIN:")
print(train_df[LABEL_COLUMN].value_counts())
print(f"\nDistribución de etiquetas en VAL:")
print(val_df[LABEL_COLUMN].value_counts())

Text column used: tweet
Train size: 5154 | Val size: 910 | Test size: 934

Distribución de etiquetas en TRAIN:
task1
NO     2862
YES    2292
Name: count, dtype: int64

Distribución de etiquetas en VAL:
task1
NO     505
YES    405
Name: count, dtype: int64


## Carga del modelo LLM

In [5]:
# Cargar el tokenizer de Mistral
# Mistral-3 usa el tokenizer v3 de Mistral
tokenizer = MistralTokenizer.v3(is_tekken=True)

# El modelo viene pre-cuantizado en FP8, usamos su configuración nativa
model = Mistral3ForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=FineGrainedFP8Config(dequantize=True),
    token=HF_TOKEN
)
model.eval()
print("Modelo cargado correctamente.")


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/531 [00:00<?, ?it/s]

Modelo cargado correctamente.


## Definición del prompt y función de inferencia

Construimos un prompt de clasificación binaria. El modelo debe responder únicamente `YES` o `NO`.

In [6]:
SYSTEM_PROMPT = (
    "You are a text classification assistant. "
    "Your task is to determine whether the following text contains sexism. "
    "Answer with exactly one word: YES or NO."
)

def build_messages(text: str):
    return [
        SystemMessage(content=SYSTEM_PROMPT),
        UserMessage(content=f"Text: {text}\n\nDoes this text contain sexism?"),
    ]

def predict_single(text: str) -> str:
    """Devuelve 'YES' o 'NO' para un texto dado."""
    messages = build_messages(text)
    
    # Tokenize using mistral_common
    tokenized = tokenizer.encode_chat_completion(
        ChatCompletionRequest(messages=messages)
    )
    input_ids = torch.tensor([tokenized.tokens]).to("cuda")

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=5,
            do_sample=False,
        )[0]

    new_tokens = output_ids[input_ids.shape[-1]:]
    decoded = tokenizer.decode(new_tokens.tolist()).strip().upper()

    if "YES" in decoded:
        return "YES"
    elif "NO" in decoded:
        return "NO"
    else:
        print(f"[WARN] Respuesta inesperada: '{decoded}' → se asigna NO")
        return "NO"

def predict_batch(texts: list, verbose: bool = True) -> list:
    """Infiere sobre una lista de textos y devuelve una lista de 'YES'/'NO'."""
    preds = []
    for i, text in enumerate(texts):
        pred = predict_single(text)
        preds.append(pred)
        if verbose and (i + 1) % 50 == 0:
            print(f"  Procesados {i+1}/{len(texts)}...")
    return preds

## Inferencia en DEV (validación)

In [7]:
print("Realizando inferencia en DEV...")
dev_preds_str = predict_batch(val_df[TEXT_COLUMN].tolist())
dev_preds = np.array([label_map[p] for p in dev_preds_str])
y_true_dev = val_df["label"].values

precision, recall, f1, _ = precision_recall_fscore_support(
    y_true_dev, dev_preds, average='binary', zero_division=0
)
acc = accuracy_score(y_true_dev, dev_preds)

print(f"\nMétricas en DEV:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

Realizando inferencia en DEV...


/home/alumno.upv.es/scheng1/.conda/envs/RFA2526pt/lib/python3.12/site-packages/mistral_common/tokens/tokenizers/tekken.py:489: FutureWarning: Using the tokenizer's special token policy (SpecialTokenPolicy.IGNORE) is deprecated. It will be removed in 1.10.0. Please pass a special token policy explicitly. Future default will be SpecialTokenPolicy.IGNORE.
  warnings.warn(


  Procesados 50/910...


  Procesados 100/910...


  Procesados 150/910...


...

  Procesados 750/910...


  Procesados 800/910...


  Procesados 850/910...


  Procesados 900/910...



Métricas en DEV:
Accuracy:  0.8143
Precision: 0.7500
Recall:    0.8741
F1-Score:  0.8073


## Evaluación en DEV con PyEvALL

In [8]:
dev_preds_for_pyevall = [
    {'test_case': 'EXIST2025', 'id': str(id_exist), 'value': pred}
    for id_exist, pred in zip(val_df['id_EXIST'].values, dev_preds_str)
]

dev_preds_df = pd.DataFrame(dev_preds_for_pyevall)
dev_preds_path = os.path.join(PREDICTIONS_DIR, 'dev_predictions_temp.json')
with open(dev_preds_path, 'w', encoding='utf-8') as f:
    f.write(dev_preds_df.to_json(orient='records'))

dev_gold = [
    {'test_case': 'EXIST2025', 'id': str(id_exist), 'value': label}
    for id_exist, label in zip(val_df['id_EXIST'].values, val_df[LABEL_COLUMN].values)
]

dev_gold_df = pd.DataFrame(dev_gold)
dev_gold_path = os.path.join(PREDICTIONS_DIR, 'dev_gold_temp.json')
with open(dev_gold_path, 'w', encoding='utf-8') as f:
    f.write(dev_gold_df.to_json(orient='records'))

evaluator = PyEvALLEvaluation()
metrics = [
    MetricFactory.Accuracy.value,
    MetricFactory.FMeasure.value,
]
report = evaluator.evaluate(dev_preds_path, dev_gold_path, metrics)
print("\n=== Evaluación en DEV con PyEvALL ===")
report.print_report()

2026-02-26 16:35:15,252 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['Accuracy', 'FMeasure']


2026-02-26 16:35:15,302 - pyevall.metrics.metrics - INFO -             evaluate() - Executing accuracy evaluation method


2026-02-26 16:35:15,417 - pyevall.metrics.metrics - INFO -             evaluate() - Executing fmeasure evaluation method



=== Evaluación en DEV con PyEvALL ===
{
  "metrics": {
    "Accuracy": {
      "name": "Accuracy",
      "acronym": "Acc",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average": 0.8142857142857143
        }],
        "average_per_test_case": 0.8142857142857143
      }
    },
    "FMeasure": {
      "name": "F-Measure",
      "acronym": "F1",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "classes": {
            "NO": 0.8207847295864262,
            "YES": 0.8072976054732042
          },
          "average": 0.8140411675298151
        }],
        "average_per_test_case": 0.8140411675298151
      }
    }
  },
  "files": {
    "dev_predictions_temp.json": {
      "name": "dev_predictions_temp.json",
      "status": "OK",
      "gold": false,
      "description": "Use parameter: report=\"embedd

## Inferencia en TEST y generación de predicciones finales

In [9]:
print("Realizando inferencia en TEST...")
test_preds_str = predict_batch(test_df[TEXT_COLUMN].tolist())
test_preds = np.array([label_map[p] for p in test_preds_str])

print(f"\nPredicciones en TEST:")
print(f"Total muestras:    {len(test_preds)}")
print(f"Predicciones YES:  {np.sum(test_preds == 1)} ({100*np.mean(test_preds == 1):.2f}%)")
print(f"Predicciones NO:   {np.sum(test_preds == 0)} ({100*np.mean(test_preds == 0):.2f}%)")

Realizando inferencia en TEST...


/home/alumno.upv.es/scheng1/.conda/envs/RFA2526pt/lib/python3.12/site-packages/mistral_common/tokens/tokenizers/tekken.py:489: FutureWarning: Using the tokenizer's special token policy (SpecialTokenPolicy.IGNORE) is deprecated. It will be removed in 1.10.0. Please pass a special token policy explicitly. Future default will be SpecialTokenPolicy.IGNORE.
  warnings.warn(


  Procesados 50/934...


  Procesados 100/934...


  Procesados 150/934...


...

  Procesados 750/934...


  Procesados 800/934...


  Procesados 850/934...


  Procesados 900/934...



Predicciones en TEST:
Total muestras:    934
Predicciones YES:  496 (53.10%)
Predicciones NO:   438 (46.90%)


## Guardar predicciones en formato PyEvALL para TEST

In [10]:
MODEL_ID = "Mistral3B"

test_preds_for_submission = [
    {'test_case': 'EXIST2025', 'id': str(id_exist), 'value': pred}
    for id_exist, pred in zip(test_df['id_EXIST'].values, test_preds_str)
]

test_preds_df = pd.DataFrame(test_preds_for_submission)

output_filename = f"{GROUP_ID}_{MODEL_ID}.json"
output_path = os.path.join(PREDICTIONS_DIR, output_filename)

with open(output_path, 'w', encoding='utf-8') as output_file:
    output_file.write(test_preds_df.to_json(orient='records'))

print(f"\nPredicciones guardadas en: {output_path}")


Predicciones guardadas en: ../results_v2/Ministral3B/predictions/BeingChillingWeWillWin_Mistral3B.json
